In [6]:
import cv2
import os
import numpy as np

# List of subjects
subjects = ["", "Donald Trump", "Johnny Depp", "Narendra Modi"]

# Function to detect face using OpenCV
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    if len(faces) == 0:
        return None, None
    (x, y, w, h) = faces[0]
    return gray[y:y+h, x:x+w], faces[0]

# Function to prepare training data
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    for dir_name in dirs:
        if not dir_name.startswith("s"):
            continue
        label = int(dir_name.replace("s", ""))
        subject_dir_path = os.path.join(data_folder_path, dir_name)
        subject_images_names = os.listdir(subject_dir_path)
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue
            image_path = os.path.join(subject_dir_path, image_name)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Unable to load image '{image_path}'")
                continue
            face, rect = detect_face(image)
            if face is not None:
                face = cv2.resize(face, (200, 300))
                faces.append(face)
                labels.append(label)
    return faces, labels

# Function to draw rectangle on image
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Function to draw text on image
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

# Function to recognize the person in the image
def predict(test_img, true_label):
    img = test_img.copy()
    face, rect = detect_face(img)
    if face is None:
        print("No face detected.")
        return img, False
    face = cv2.resize(face, (200, 300))
    label, confidence = face_recognizer.predict(face)
    label_text = subjects[label]
    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1]-5)
    return img, (label == true_label)

# Create directory to save analyzed images
output_folder_path = "Output file storage path"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

print("Preparing data...")
faces, labels = prepare_training_data("Your file path for training data")
print("Data prepared")

print("Total faces:", len(faces))
print("Total labels:", len(labels))

face_recognizer = cv2.face.EigenFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

print("Predicting images...")

# Path to the test data folder
test_data_folder_path = "Your File Pathfor testing data"

# Get list of test images
test_images_names = os.listdir(test_data_folder_path)

# Initialize counters for accuracy calculation per label
label_counts = {i: {'correct': 0, 'total': 0} for i in range(1, len(subjects))}

# Loop through each test image
for image_name in test_images_names:
    if image_name.startswith("."):
        continue
    image_path = os.path.join(test_data_folder_path, image_name)
    test_img = cv2.imread(image_path)
    if test_img is not None:
        true_label = int(image_name.split("_")[0].replace("s", ""))  # Assuming test images are named as 'sX_image.jpg'
        predicted_img, correct = predict(test_img, true_label)
        if correct:
            label_counts[true_label]['correct'] += 1
        label_counts[true_label]['total'] += 1

        # Save the analyzed image
        output_image_path = os.path.join(output_folder_path, image_name)
        cv2.imwrite(output_image_path, predicted_img)

        cv2.imshow(image_name, cv2.resize(predicted_img, (400, 500)))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    else:
        print(f"Error: Unable to load test image '{image_path}'")

# Print summary of the prediction results per label
for label, counts in label_counts.items():
    if counts['total'] > 0:
        accuracy_percentage = (counts['correct'] / counts['total']) * 100
        print(f"Prediction summary for {subjects[label]}: {counts['correct']}/{counts['total']} ({accuracy_percentage:.2f}%)")
    else:
        print(f"No test images found for {subjects[label]}")

print("Prediction complete")


Preparing data...
Data prepared
Total faces: 50
Total labels: 50
Predicting images...
No face detected.
No face detected.
No face detected.
No face detected.
Prediction summary for Donald Trump: 18/20 (90.00%)
Prediction summary for Johnny Depp: 13/20 (65.00%)
Prediction summary for Narendra Modi: 5/20 (25.00%)
Prediction complete
